In [37]:
import duckdb
import pandas as pd
import os

PAI_DIR = os.getcwd()  # diretório atual para notebooks jupyter
DATA_DIR = os.path.join(PAI_DIR, "data", "processed")

In [38]:
con = duckdb.connect(database=':memory:')

# Cria tabela no DuckDB a partir do parquet
con.execute(f"""
    CREATE TABLE estimativa_cre AS 
    SELECT * FROM '{DATA_DIR}/estimativa_cre_16-06-25.parquet'
""")

# Consulta inicial (pode ser LIMIT 10 para preview)
df_duck = con.execute("SELECT * FROM estimativa_cre LIMIT 10").df()

df_duck.sample(2)

,ug_codigo,descricao_ug,fonte_recurso,descricao_fonte,classificacao_ajustada,classificacao_intra,nomenclatura_receita,rec_grupo,rec_subgrupo,metodo_principal,ano,mes,valor
5,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125201.0,None,Imposto sobre Transmissão “Causa Mortis” e Doa...,Receita Bruta,Receita Estimada,MQO,2023,1,1622645.15
8,140001,SECRETARIA DE ESTADO DE FINANCAS,501.0,Outros Recursos não Vinculados,11125203.0,None,ITCMD Dívida Ativa - Estado,Receita Bruta,Receita Estimada,NÃO PROJETAR,2023,1,15256.33


In [24]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN"),
)

def consulta_ia(pergunta: str):
    prompt = f"""
    Você é um assistente que gera consultas SQL para DuckDB.
    A tabela disponível se chama 'estimativa_cre' e tem as seguintes colunas:
    {', '.join(df_duck.columns.tolist())}

    Pergunta do usuário: {pergunta}

    Responda apenas com a SQL.
    """

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0,
    )

    query = response.choices[0].message.content.strip()

    # Executa a query no DuckDB
    try:
        resultado = con.execute(query).fetchdf()
        return query, resultado
    except Exception as e:
        return query, f"Erro ao executar SQL: {e}"

In [31]:
query, resultado = consulta_ia("O total arrecadado por ano da fonte 500")
print("Query usada:\n", query)
print("\nResultado:\n", resultado)

Query usada:
 SELECT SUM(valor) AS total_arrecadado FROM estimativa_cre WHERE fonte_recurso = 500 GROUP BY ano

Resultado:
    total_arrecadado
0      8.622825e+09
1      1.002981e+10
2      1.069943e+10
3      1.127532e+10
4      1.216892e+10


In [39]:
# Consulta SQL
resultado = con.execute("""
    SELECT 
        mes, SUM(valor) AS total_arrecadado 
    FROM estimativa_cre 
    WHERE fonte_recurso = 500 
    AND ano = '2025'
    GROUP BY mes
""").df()

resultado

,mes,total_arrecadado
0,1,8.729471e+08
1,2,9.823469e+08
2,3,8.355140e+08
3,4,8.609253e+08
4,5,9.494757e+08
5,6,9.387981e+08
6,7,7.862592e+08
7,8,8.799675e+08
8,9,7.925028e+08
9,10,8.150094e+08
